## ID : 816000325
## Name: Ajay Sieunarine
## Email: ajay.sieunarine@my.uwi.edu
## Repo: https://github.com/jefroy/big-data-A4

In [95]:
# math and graph stuff
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# text analysis stuff
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import string
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from string import punctuation

# ML stuff
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score
from sklearn import svm
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

# utility stuff
from sklearn.utils import shuffle
from itertools import chain
from sklearn.preprocessing import LabelEncoder


sep = '============================================================================================'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\idisc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\idisc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Warm Up (20 marks) 

In [96]:
# documents/sentences
corpus = [ 
    'Apple Orange Orange Apple',
    'Apple Banana Apple Banana',
    'Banana Apple Banana Banana Banana Apple',
    'Banana Orange Banana Banana Orange Banana',
    'Banana Apple Banana Banana Orange Banana'
]

def corp2vec(corpus):
    '''
    This function converts a document to a vector showing its term frequencies.
    Each column represents a term in the corpus. (in this case, there are 3 terms).
    Uses CountVectorizer() as a model.
    '''
    tf_vectorizer = CountVectorizer() # model
    tf = tf_vectorizer.fit_transform(corpus) # get term frequencies
    return tf.A # return the array representation

print(type(corp2vec(corpus)))
print(corp2vec(corpus))

<class 'numpy.ndarray'>
[[2 0 2]
 [2 2 0]
 [2 4 0]
 [0 4 2]
 [1 4 1]]


# Preprocessing and Data Organization (20 marks) 

In [97]:
df = pd.read_csv('MrTrumpSpeeches.csv', sep='~', encoding="ISO-8859-1")

# remove this line please
df = df[:10]

print(df.shape)
df.head()

(10, 9)


,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...


In [98]:
df.dtypes

id                 object
playlist           object
upload_date         int64
title              object
view_count        float64
average_rating    float64
like_count        float64
dislike_count     float64
subtitles          object
dtype: object

## Task 1: 
Create a new column in the dataframe called 'sentiment'. Using appropriate existing columns, populate the new column with 0's and 1's where 0 refers to a negative sentiment and 1 refers to a positive sentiment. 

- using the `average_rating` column, find the average score
    * ratings above or equal to this average are denoted by sentiment = 1
    * ratings under this average are denoted by sentiment = 0

In [99]:
# find the avg/mid score possible:
max_score = max(df['average_rating'])
min_score = min(df['average_rating'])
avg_score = (max_score + min_score) / 2
print(max_score)
print(min_score)
print(avg_score)

4.92473125458
4.25925922394
4.59199523926


In [100]:
def pruneRating(x):
    '''
    utility function for lambda
    returns 1 for good/avg rating
    returns 0 for bad rating
    '''
    if x >= avg_score:
        return 1
    if x < avg_score:
        return 0

df['sentiment'] = df['average_rating'].apply(
    lambda x: pruneRating(x)
)

enc3 = LabelEncoder()
enc3.fit(df['sentiment'])

print(enc3.classes_)
print(sep)
print(df['sentiment'].describe()) 
print(sep)
df.head()

[0 1]
count    10.000000
mean      0.400000
std       0.516398
min       0.000000
25%       0.000000
50%       0.000000
75%       1.000000
max       1.000000
Name: sentiment, dtype: float64


,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,sentiment
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,0
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...,0
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...,1
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...,0
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...,1


## Task 2
- Clean the subtitles data and store the cleaned text in a new column 'subtitle_clean'.
    * For each step of your text cleaning give a brief explanation of why you chose to perform that method on the text. 

In [101]:
# this block of code will take a little while to process since the text is being thoroughly cleaned.
wnl = WordNetLemmatizer()
porter = PorterStemmer()
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation.
    2. Remove all stopwords.
    4. Convert all remaining words to lowercase.
        - if the string has numbers in it, discard it.
    5. Lemmatize.
    6. Convert to the stem of the word.
    7. Returns a list of the cleaned text.
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    # convert all words to lowercase and lemmatize and stem
    
    arr = []
    for word in words:
        word = word.lower()
        word = porter.stem(word)
        word = wnl.lemmatize(word)
        if word.isalpha():
            word += ' '
            arr.append(word)
        
    arr = ''.join(arr) # convert the array of words/tokens into a string
#     print(arr,sep)
    if len(arr) > 0:
        return arr
    else:
        return "smeckledorf" # if the entire subtitle had no words in it.

def test_text_process():
    '''
    function to hold some test cases to ensure the lambda function does its job
    '''
    print(
        'Test stopwords and lowercase:\nHI I AM A STOPWORD = ', 
        text_process("HI I AM A STOPWORD")
    )
    print(sep)
    print(
        'Test Stemming words and removing punctuation:\nfishing, fisher, fished = ', 
        text_process("fishing, fisher , fished")
    )

def pruneSubtitle(x):
    '''
    utility function for lambda
    x - string from `subtitles` col
    '''
    return text_process(x)

# test_text_process()

df['subtitle_clean'] = df['subtitles'].apply(
    lambda x: pruneSubtitle(x)
)

df.head()

,id,playlist,upload_date,title,view_count,average_rating,like_count,dislike_count,subtitles,sentiment,subtitle_clean
0,-2WTNSujhjk,Donald Trump Speeches & Events,20160220,Live Stream: Donald Trump Victory Rally in Spa...,4057.0,4.259259,44.0,10.0,presidents of the United States mr. go tr...,0,presid unit state mr go trapp famili thank app...
1,-64nfy6i58w,Donald Trump Speeches & Events,20161107,LAST RALLY: Donald Trump FINAL CAMPAIGN Rally ...,47276.0,4.358025,952.0,182.0,it's now officially Tuesday November a di...,0,offici tuesday novemb ever think youd hear maj...
2,-7Sp31hTxkU,Donald Trump Speeches & Events,20160423,"FULL SPEECH: Donald Trump Rally in Bridgeport,...",19966.0,4.666667,220.0,20.0,you [Music] [Music] [Music] you I...,1,music music music great countri leader free un...
3,-byuyavcNI4,Donald Trump Speeches & Events,20160617,"Full Speech: Donald Trump Rally in Houston, Te...",15138.0,4.582491,266.0,31.0,we welcome stars and president [Music] ...,0,welcom star presid music place live place wow ...
4,09BXh-AA72M,Donald Trump Speeches & Events,20161105,"Full Speech: Donald Trump Rally in Denver, Col...",8720.0,4.924731,365.0,7.0,you thank you [Music] great people Gr...,1,thank music great peopl granna three day go wi...


I chose to Lemmatize AND Stem each word since the amount of text being processed is very large, and this will also allow the models to get better results.

## Task 3
Use TFIDFVectorizer and CountVectorizer to encode the cleaned subtitles. 

In [102]:
# # define the data to use
clean_subs = list(df['subtitle_clean']) 
# clean_subs = list(df['subtitle_clean']) # this line might be crashing the file 
# # clean_subs = df['subtitle_clean'].values # kinda pointless, parsing as list() is neater
# # jupyter might require more memory to run this

print(type(clean_subs))
print(sep)
print(len(clean_subs))

<class 'list'>
10


In [103]:
# CountVectorizer
tf_vectorizer = CountVectorizer()
tf = tf_vectorizer.fit_transform(clean_subs)
print("CountVectorizer:")
print(tf)
print(sep)
print(tf_vectorizer.get_feature_names())

CountVectorizer:
  (0, 1948)	3
  (0, 2703)	2
  (0, 2414)	4
  (0, 1675)	2
  (0, 1073)	62
  (0, 2638)	1
  (0, 911)	6
  (0, 2550)	16
  (0, 113)	7
  (0, 1681)	6
  (0, 862)	3
  (0, 2809)	9
  (0, 2784)	16
  (0, 215)	2
  (0, 1858)	24
  (0, 2376)	7
  (0, 381)	7
  (0, 2384)	5
  (0, 1427)	10
  (0, 1060)	11
  (0, 1504)	1
  (0, 286)	1
  (0, 1451)	3
  (0, 2802)	3
  (0, 1892)	3
  :	:
  (9, 38)	1
  (9, 1956)	1
  (9, 2077)	1
  (9, 558)	1
  (9, 2678)	1
  (9, 2405)	1
  (9, 1273)	1
  (9, 1446)	1
  (9, 2135)	1
  (9, 1708)	1
  (9, 1002)	1
  (9, 941)	1
  (9, 1218)	1
  (9, 182)	1
  (9, 1376)	1
  (9, 2139)	1
  (9, 2637)	1
  (9, 1303)	1
  (9, 1199)	1
  (9, 1536)	1
  (9, 1437)	1
  (9, 2133)	1
  (9, 2744)	1
  (9, 964)	1
  (9, 76)	1
['abandon', 'abat', 'abl', 'abolish', 'absolut', 'abus', 'academi', 'accessori', 'accid', 'accomplish', 'accord', 'account', 'accus', 'achiev', 'acknowledg', 'acquit', 'across', 'act', 'action', 'activ', 'actual', 'acumen', 'ad', 'add', 'addict', 'addit', 'address', 'adelson', 'admini

In [104]:
# TFIDF 
tfidf_vec = TfidfVectorizer()
tfidf = tfidf_vec.fit_transform(clean_subs)
print("TFIDF:")
print(tfidf)
print(sep)
print(tfidf_vec.get_feature_names())

TFIDF:
  (0, 1480)	0.04861311375377568
  (0, 1185)	0.01064928767329923
  (0, 1715)	0.04861311375377568
  (0, 1928)	0.015615852504595133
  (0, 1418)	0.012818828241991058
  (0, 1102)	0.03477699244126626
  (0, 1612)	0.01738849622063313
  (0, 1120)	0.012818828241991058
  (0, 1086)	0.01738849622063313
  (0, 2673)	0.009722622750755137
  (0, 2833)	0.011673668219865305
  (0, 859)	0.026297245336658864
  (0, 2862)	0.011673668219865305
  (0, 1587)	0.01064928767329923
  (0, 2816)	0.019445245501510274
  (0, 2554)	0.11177530525964517
  (0, 1434)	0.015615852504595133
  (0, 108)	0.031231705009190265
  (0, 594)	0.022355061051929033
  (0, 1222)	0.011673668219865305
  (0, 1674)	0.03502100465959591
  (0, 2437)	0.011673668219865305
  (0, 552)	0.01411710359786226
  (0, 475)	0.01955803678932496
  (0, 1548)	0.03477699244126626
  :	:
  (9, 994)	0.0048003700783518415
  (9, 372)	0.06720518109692579
  (9, 1716)	0.052804070861870264
  (9, 2086)	0.042063153709174864
  (9, 1892)	0.0048003700783518415
  (9, 2802)	0.0